<a href="https://colab.research.google.com/github/viethoang1122334/Machine-Learning/blob/master/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents
1. [Introduction](#introduction)

2. [Dataset Analysis](#question1)

3. [Apply models in Naïve Bayes, Logistic Regression, DecisionTree, Random Forest, Gradient Boosting and SVM](#question2)

4. [Apply PCA, SelectKBest and RFE for feature selections](#question3)

5. [Using gridsearchCV to obtain best parameters for models](#question4)

6. [Compare performances among models, write up analysis why the model is good or bad in the algorithmic approach](#question5)

7. [Conclusions](#question6)

## 1. Introduction <a name="introduction"></a>
This project use dataset from http://stat-computing.org/dataexpo/2009/the-data.html to predict how late flights
will be. A flight only counts as late if it is more than 30 minutes late. The project will answer 6 following questions:
1. Dataset Analysis
2. Apply models in Naïve Bayes, Logistic Regression, DecisionTree, Random Forest, Gradient Boosting and SVM.
3. Apply PCA, SelectKBest and RFE for feature selections.
4. Using gridsearchCV to obtain best parameters for models.
5. Compare performances among models, write up analysis why the model is good or bad in the algorithmic approach (explain why the algorithm is good or bad for the dataset structure, can you do something to improve the model?)
6. Include the conclusions.

## 2. Dataset Analysis<a name="dataset"></a>

#### Import Python libraries<a name="library"></a>

In [0]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn 
#import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

#### Import data<a name="data"></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
url = 'gdrive/My Drive/FlightDelay/delay.bz2'
data = pd.read_csv(url)

#### Data description<a name="description"></a>

In [92]:
# Dataset head
pd.set_option('display.max_columns', None)  
data.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


#### Check rows with Cancelled = 1

In [98]:
data.loc[data['Cancelled'] == 1]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
178,2008,1,3,4,NaN,700,NaN,830,WN,126,NaN,NaN,90.0,NaN,NaN,NaN,LAS,OAK,407,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
373,2008,1,3,4,NaN,1100,NaN,1215,WN,1146,NaN,NaN,75.0,NaN,NaN,NaN,LAX,OAK,337,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
399,2008,1,3,4,NaN,905,NaN,1025,WN,469,NaN,NaN,80.0,NaN,NaN,NaN,LAX,SFO,337,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
401,2008,1,3,4,NaN,1620,NaN,1740,WN,618,NaN,NaN,80.0,NaN,NaN,NaN,LAX,SFO,337,NaN,NaN,1,C,0,NaN,NaN,NaN,NaN,NaN
415,2008,1,3,4,NaN,1930,NaN,2035,WN,2528,NaN,NaN,65.0,NaN,NaN,NaN,LAX,SJC,308,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009455,2008,12,13,6,NaN,600,NaN,815,DL,1211,N3760C,NaN,135.0,NaN,NaN,NaN,LGA,CVG,585,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
7009464,2008,12,13,6,NaN,1930,NaN,2129,DL,1218,N3744D,NaN,119.0,NaN,NaN,NaN,CVG,LGA,585,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
7009564,2008,12,13,6,NaN,700,NaN,1035,DL,1421,N950DL,NaN,215.0,NaN,NaN,NaN,BOS,FLL,1237,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
7009565,2008,12,13,6,NaN,1115,NaN,1432,DL,1422,N950DL,NaN,197.0,NaN,NaN,NaN,FLL,BOS,1237,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN


- DepTime, ArrTime, ActualElapsedTime, AirTime, ArrDelay, DepDelay, TaxiIn, TaxiOut are NaN.
- Diverted is 0 in these rows.
- CancellationCode has information  in these columns.

#### Check rows with Deverted =1

In [99]:
data.loc[data['Diverted'] == 1]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1763,2008,1,3,4,922.0,915,NaN,1050,WN,1069,N630WN,NaN,95.0,NaN,NaN,7.0,SAN,SMF,480,NaN,12.0,0,NaN,1,NaN,NaN,NaN,NaN,NaN
1911,2008,1,3,4,2325.0,1900,NaN,2030,WN,2092,N302SW,NaN,90.0,NaN,NaN,265.0,SFO,SAN,447,NaN,11.0,0,NaN,1,NaN,NaN,NaN,NaN,NaN
2651,2008,1,4,5,1949.0,1905,NaN,1910,WN,1403,N504SW,NaN,65.0,NaN,NaN,44.0,BOI,RNO,335,NaN,11.0,0,NaN,1,NaN,NaN,NaN,NaN,NaN
2726,2008,1,4,5,737.0,705,NaN,825,WN,178,N718SW,NaN,80.0,NaN,NaN,32.0,BUR,SMF,358,NaN,13.0,0,NaN,1,NaN,NaN,NaN,NaN,NaN
3383,2008,1,4,5,931.0,930,NaN,1145,WN,2011,N726SW,NaN,255.0,NaN,NaN,1.0,HOU,OAK,1642,NaN,10.0,0,NaN,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7006200,2008,12,10,3,714.0,640,1153.0,859,DL,1610,N956DL,NaN,79.0,NaN,NaN,34.0,JAN,ATL,341,16.0,10.0,0,NaN,1,NaN,NaN,NaN,NaN,NaN
7006401,2008,12,11,4,1355.0,1106,7.0,1950,DL,26,N3747D,NaN,344.0,NaN,NaN,169.0,LAX,JFK,2475,13.0,17.0,0,NaN,1,NaN,NaN,NaN,NaN,NaN
7007034,2008,12,11,4,1527.0,1520,2106.0,1708,DL,1102,N924DL,NaN,108.0,NaN,NaN,7.0,IAD,ATL,533,9.0,19.0,0,NaN,1,NaN,NaN,NaN,NaN,NaN
7007389,2008,12,11,4,717.0,725,NaN,945,DL,1491,N976DL,NaN,200.0,NaN,NaN,-8.0,LGA,MSY,1183,NaN,25.0,0,NaN,1,NaN,NaN,NaN,NaN,NaN


- ArrTime, ActualElapsedTime, AirTime, ArrDelay, TaxiIn are mostly  NaN, these variables are related to diversion so this is usual.

#### Count values of CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay

In [116]:
data['CarrierDelay'].value_counts() 

0.0       854113
6.0        22799
7.0        21501
1.0        21272
2.0        20981
           ...  
608.0          1
607.0          1
867.0          1
868.0          1
1542.0         1
Name: CarrierDelay, Length: 984, dtype: int64

In [117]:
data['WeatherDelay'].value_counts() 

0.0      1424750
10.0        2870
6.0         2739
15.0        2724
7.0         2528
          ...   
528.0          1
527.0          1
523.0          1
519.0          1
915.0          1
Name: WeatherDelay, Length: 599, dtype: int64

In [118]:
data['NASDelay'].value_counts() 

0.0       596704
15.0       39584
16.0       35246
1.0        34065
2.0        33178
           ...  
506.0          1
505.0          1
504.0          1
503.0          1
1207.0         1
Name: NASDelay, Length: 574, dtype: int64

In [119]:
data['SecurityDelay'].value_counts() 

0.0      1518533
7.0          350
6.0          327
8.0          286
15.0         286
          ...   
132.0          1
134.0          1
135.0          1
136.0          1
175.0          1
Name: SecurityDelay, Length: 156, dtype: int64

In [120]:
data['LateAircraftDelay'].value_counts() 

0.0      825317
15.0      19069
16.0      17678
17.0      16738
18.0      15856
          ...  
514.0         1
512.0         1
507.0         1
501.0         1
661.0         1
Name: LateAircraftDelay, Length: 564, dtype: int64

In [4]:
# Number of rows and columns
print("Shape of dataframe is: {}".format(data.shape))

Shape of dataframe is: (7009728, 29)


In [5]:
# Dataset columns
data.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

In [6]:
# Data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009728 entries, 0 to 7009727
Data columns (total 29 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtypes: float64(14), int64(10), object(5)
memory usage: 1.5+ GB


In [7]:
data.isnull().sum()

Year                       0
Month                      0
DayofMonth                 0
DayOfWeek                  0
DepTime               136246
CRSDepTime                 0
ArrTime               151649
CRSArrTime                 0
UniqueCarrier              0
FlightNum                  0
TailNum                83365
ActualElapsedTime     154699
CRSElapsedTime           844
AirTime               154699
ArrDelay              154699
DepDelay              136246
Origin                     0
Dest                       0
Distance                   0
TaxiIn                151649
TaxiOut               137058
Cancelled                  0
CancellationCode     6872294
Diverted                   0
CarrierDelay         5484993
WeatherDelay         5484993
NASDelay             5484993
SecurityDelay        5484993
LateAircraftDelay    5484993
dtype: int64

In [8]:
# Percentage of missing values
data.isnull().sum()/len(data)*100

Year                  0.000000
Month                 0.000000
DayofMonth            0.000000
DayOfWeek             0.000000
DepTime               1.943670
CRSDepTime            0.000000
ArrTime               2.163408
CRSArrTime            0.000000
UniqueCarrier         0.000000
FlightNum             0.000000
TailNum               1.189276
ActualElapsedTime     2.206919
CRSElapsedTime        0.012040
AirTime               2.206919
ArrDelay              2.206919
DepDelay              1.943670
Origin                0.000000
Dest                  0.000000
Distance              0.000000
TaxiIn                2.163408
TaxiOut               1.955254
Cancelled             0.000000
CancellationCode     98.039382
Diverted              0.000000
CarrierDelay         78.248300
WeatherDelay         78.248300
NASDelay             78.248300
SecurityDelay        78.248300
LateAircraftDelay    78.248300
dtype: float64

#### Remove norelated columns

- Year is a variable indicating the year of the survey, should be removed.
- DepTime, ArrTime, ActualElapsedTime, TaxiIn, TaxiOut are the variables that appear after the flight departs.
- CRSElapsedTime can be inferred from DepTime and Arrtime (CRSElapsedTime=Arrtime-DepTime)
- TailNum,FlightNum,UniqueCarrier are variables unrelated to flight delay
- Cancelled are removed because non-canceled flights have Cancelled = 1. CancellationCode is also removed
- Diverted are removed because rows with "Diverted = 1" seem to have missing values and I will remove them later. The remaining columns with "Devert = 1" have no sense because both delay and non-delay flights are included in it.
- CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay are removed because they are not related to delays.
- Origin, Dest are removed to speed up model training.



In [9]:
# To remove an entire column
remove = ['Year','DepTime','ArrTime','ActualElapsedTime','CRSElapsedTime','AirTime','TailNum','FlightNum',
          'UniqueCarrier','TaxiIn','TaxiOut','Cancelled','CancellationCode','Diverted','CarrierDelay',
          'WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay',
          'Origin','Dest']
data1= data.drop(remove, axis=1)
data1

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,ArrDelay,DepDelay,Distance
0,1,3,4,1955,2225,-14.0,8.0,810
1,1,3,4,735,1000,2.0,19.0,810
2,1,3,4,620,750,14.0,8.0,515
3,1,3,4,930,1100,-6.0,-4.0,515
4,1,3,4,1755,1925,34.0,34.0,515
...,...,...,...,...,...,...,...,...
7009723,12,13,6,959,1150,14.0,3.0,533
7009724,12,13,6,835,1023,-2.0,-1.0,874
7009725,12,13,6,700,856,0.0,-5.0,545
7009726,12,13,6,1240,1437,9.0,11.0,533


In [10]:
# Percentage of missing values
data1.isnull().sum()/len(data)*100

Month         0.000000
DayofMonth    0.000000
DayOfWeek     0.000000
CRSDepTime    0.000000
CRSArrTime    0.000000
ArrDelay      2.206919
DepDelay      1.943670
Distance      0.000000
dtype: float64

#### Remove rows misssing data

In [11]:
# Remove rows missing data
data2 = data1.dropna()
# Confirm that observations missing data were removed  
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6855029 entries, 0 to 7009727
Data columns (total 8 columns):
Month         int64
DayofMonth    int64
DayOfWeek     int64
CRSDepTime    int64
CRSArrTime    int64
ArrDelay      float64
DepDelay      float64
Distance      int64
dtypes: float64(2), int64(6)
memory usage: 470.7 MB


#### Convert ArrDelay, DepDelay to 0 and 1

In [12]:
# Count the number of rows for each type
data2.groupby('DepDelay').size()

DepDelay
-534.0     1
-92.0      1
-79.0      1
-71.0      1
-70.0      3
          ..
 1597.0    1
 1710.0    1
 1952.0    1
 2457.0    1
 2467.0    1
Length: 1129, dtype: int64

In [13]:
# Count the number of rows for each type
data2.groupby('ArrDelay').size()

ArrDelay
-519.0     1
-129.0     1
-109.0     1
-92.0      1
-91.0      1
          ..
 1655.0    1
 1707.0    1
 1951.0    1
 2453.0    1
 2461.0    1
Length: 1153, dtype: int64

In [14]:
data2['OutputDep']=(data2.DepDelay>30).astype("float64")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
def calc_prevalence(y_actual):
    return (sum(y_actual)/len(y_actual))

In [16]:
print('Prevalence:%.3f'%calc_prevalence(data2['OutputDep'].values))

Prevalence:0.118


In [17]:
data2['OutputArr']=(data2.ArrDelay>30).astype("float64")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
print('Prevalence:%.3f'%calc_prevalence(data2['OutputArr'].values))

Prevalence:0.131


In [19]:
data2

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,ArrDelay,DepDelay,Distance,OutputDep,OutputArr
0,1,3,4,1955,2225,-14.0,8.0,810,0.0,0.0
1,1,3,4,735,1000,2.0,19.0,810,0.0,0.0
2,1,3,4,620,750,14.0,8.0,515,0.0,0.0
3,1,3,4,930,1100,-6.0,-4.0,515,0.0,0.0
4,1,3,4,1755,1925,34.0,34.0,515,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
7009723,12,13,6,959,1150,14.0,3.0,533,0.0,0.0
7009724,12,13,6,835,1023,-2.0,-1.0,874,0.0,0.0
7009725,12,13,6,700,856,0.0,-5.0,545,0.0,0.0
7009726,12,13,6,1240,1437,9.0,11.0,533,0.0,0.0


#### One-hot-encoding Month, DayofMonth, DayOfWeek 

In [20]:
cols_cat = ['Month','DayofMonth','DayOfWeek']
cols_cat

['Month', 'DayofMonth', 'DayOfWeek']

In [21]:
# Count number of type
data2[cols_cat].nunique()

Month         12
DayofMonth    31
DayOfWeek      7
dtype: int64

In [22]:
# Convert 'Month', 'DayofMonth', 'DayOfWeek' to object
data2[cols_cat] = data2[cols_cat].astype('object')


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [23]:
# One-hot-encoding 'Month', 'DayofMonth', 'DayOfWeek'
data_cat = pd.get_dummies(data2[cols_cat],drop_first = True)
data_cat.head(10)

,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,DayofMonth_2,DayofMonth_3,DayofMonth_4,DayofMonth_5,DayofMonth_6,DayofMonth_7,DayofMonth_8,DayofMonth_9,DayofMonth_10,DayofMonth_11,DayofMonth_12,DayofMonth_13,DayofMonth_14,DayofMonth_15,DayofMonth_16,DayofMonth_17,DayofMonth_18,DayofMonth_19,DayofMonth_20,DayofMonth_21,DayofMonth_22,DayofMonth_23,DayofMonth_24,DayofMonth_25,DayofMonth_26,DayofMonth_27,DayofMonth_28,DayofMonth_29,DayofMonth_30,DayofMonth_31,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [0]:
# Add "one-hot-encoding" columns to data
data3 = pd.concat([data2,data_cat], axis = 1)

In [25]:
cols_all_cat = list(data_cat.columns)
print('number of categorical:',len(cols_all_cat))

number of categorical: 47


In [0]:
# Drop original 'Month','DayofMonth','DayOfWeek','ArrDelay','DepDelay' columns
data3_drop= data3.drop(['Month','DayofMonth','DayOfWeek','ArrDelay','DepDelay'], axis=1)

In [27]:
data3_drop

,CRSDepTime,CRSArrTime,Distance,OutputDep,OutputArr,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,DayofMonth_2,DayofMonth_3,DayofMonth_4,DayofMonth_5,DayofMonth_6,DayofMonth_7,DayofMonth_8,DayofMonth_9,DayofMonth_10,DayofMonth_11,DayofMonth_12,DayofMonth_13,DayofMonth_14,DayofMonth_15,DayofMonth_16,DayofMonth_17,DayofMonth_18,DayofMonth_19,DayofMonth_20,DayofMonth_21,DayofMonth_22,DayofMonth_23,DayofMonth_24,DayofMonth_25,DayofMonth_26,DayofMonth_27,DayofMonth_28,DayofMonth_29,DayofMonth_30,DayofMonth_31,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7
0,1955,2225,810,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,735,1000,810,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,620,750,515,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,930,1100,515,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,1755,1925,515,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009723,959,1150,533,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7009724,835,1023,874,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7009725,700,856,545,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7009726,1240,1437,533,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


## 3. Apply models in Naïve Bayes, Logistic Regression, DecisionTree, Random Forest, Gradient Boosting and SVM.

#### Select the first 100000 rows for training

In [28]:
data3_drop1 = data3_drop.drop(data3_drop.index[100000:6855029])
data3_drop1

,CRSDepTime,CRSArrTime,Distance,OutputDep,OutputArr,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,DayofMonth_2,DayofMonth_3,DayofMonth_4,DayofMonth_5,DayofMonth_6,DayofMonth_7,DayofMonth_8,DayofMonth_9,DayofMonth_10,DayofMonth_11,DayofMonth_12,DayofMonth_13,DayofMonth_14,DayofMonth_15,DayofMonth_16,DayofMonth_17,DayofMonth_18,DayofMonth_19,DayofMonth_20,DayofMonth_21,DayofMonth_22,DayofMonth_23,DayofMonth_24,DayofMonth_25,DayofMonth_26,DayofMonth_27,DayofMonth_28,DayofMonth_29,DayofMonth_30,DayofMonth_31,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7
0,1955,2225,810,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,735,1000,810,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,620,750,515,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,930,1100,515,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,1755,1925,515,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101319,1410,1532,373,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
101320,1410,1532,373,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
101321,1410,1532,373,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
101322,1410,1532,373,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
# Train_test_Split
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data3_drop1.drop(['OutputArr'], axis=1), data3_drop1['OutputArr'],
                                                    test_size=0.2, random_state=42)

In [30]:
# describes info about train and test set 
print("Number transactions X_train dataset: ", train_x.shape) 
print("Number transactions y_train dataset: ", train_y.shape) 
print("Number transactions X_test dataset: ", test_x.shape) 
print("Number transactions y_test dataset: ", test_y.shape)

Number transactions X_train dataset:  (80000, 51)
Number transactions y_train dataset:  (80000,)
Number transactions X_test dataset:  (20000, 51)
Number transactions y_test dataset:  (20000,)


In [31]:
print("Counts of label '1': {}".format(sum(train_y == 1))) 
print("Counts of label '0': {} \n".format(sum(train_y == 0))) 

Counts of label '1': 8784
Counts of label '0': 71216 



#### Naive Bayes

In [32]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(train_x,train_y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
# Make predictions using the testing set
y_pred = nb.predict(test_x)

In [34]:
# Print classification_report
print(metrics.classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       0.98      0.72      0.83     17812
         1.0       0.28      0.87      0.42      2188

    accuracy                           0.74     20000
   macro avg       0.63      0.80      0.63     20000
weighted avg       0.90      0.74      0.79     20000



In [35]:
# Print confusion_matrix
print(metrics.confusion_matrix(test_y, y_pred))

[[12872  4940]
 [  290  1898]]


#### Decision Tree

In [36]:
from sklearn import tree

# Create a Decision Tree Classifier
decision_tree = tree.DecisionTreeClassifier()

# Make predictions using the testing set
decision_tree.fit(train_x,train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
# Make predictions using the testing set
y_pred_tree = decision_tree.predict(test_x)

In [38]:
# Print classification_report
print(metrics.classification_report(test_y, y_pred_tree))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97     17812
         1.0       0.74      0.74      0.74      2188

    accuracy                           0.94     20000
   macro avg       0.85      0.86      0.85     20000
weighted avg       0.94      0.94      0.94     20000



In [39]:
# Print confusion_matrix
print(metrics.confusion_matrix(test_y, y_pred_tree))

[[17227   585]
 [  558  1630]]


#### Logistic Regression

In [40]:
from sklearn.linear_model import LogisticRegression

#Create a Gaussian Classifier
logisticRegr = LogisticRegression(class_weight='balanced')

#Train the model
logisticRegr.fit(train_x,train_y)

# Make predictions using the testing set
y_pred_logis = logisticRegr.predict(test_x)

THRESHOLD = 0.8
y_pred_logis_threshold = np.where(logisticRegr.predict_proba(test_x)[:,1] > THRESHOLD, 1, 0)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [41]:
# Print classification_report
print(metrics.classification_report(test_y, y_pred_logis_threshold))

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     17812
         1.0       0.84      0.86      0.85      2188

    accuracy                           0.97     20000
   macro avg       0.91      0.92      0.92     20000
weighted avg       0.97      0.97      0.97     20000



In [42]:
# Print confusion_matrix
print(metrics.confusion_matrix(test_y, y_pred_logis))

[[17462   350]
 [  314  1874]]


Random Forest

In [43]:
# Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

# Create a Gaussian Classifier
random_forest=RandomForestClassifier(class_weight="balanced")

# Train the model
random_forest.fit(train_x,train_y)

# Make predictions using the testing set
y_pred_randomforest=random_forest.predict(test_x)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [44]:
# Print classification_report
print(metrics.classification_report(test_y, y_pred_randomforest))

              precision    recall  f1-score   support

         0.0       0.97      0.98      0.98     17812
         1.0       0.85      0.78      0.82      2188

    accuracy                           0.96     20000
   macro avg       0.91      0.88      0.90     20000
weighted avg       0.96      0.96      0.96     20000



In [45]:
# Print confusion_matrix
print(metrics.confusion_matrix(test_y, y_pred_randomforest))

[[17505   307]
 [  471  1717]]


#### Gradient Boosting

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

#Create a GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.5, max_features=2, max_depth = 2, random_state = 0)

#Train the model
gb.fit(train_x,train_y)

#Test the model
y_pred_gradient=gb.predict(test_x)

In [48]:
# Print classification_report
print(metrics.classification_report(test_y, y_pred_gradient))

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     17812
         1.0       0.84      0.83      0.84      2188

    accuracy                           0.96     20000
   macro avg       0.91      0.90      0.91     20000
weighted avg       0.96      0.96      0.96     20000



In [49]:
# Print confusion_matrix
print(metrics.confusion_matrix(test_y, y_pred_randomforest))

[[17505   307]
 [  471  1717]]


#### SVM

(Slow training speed)

In [0]:
#from sklearn.svm import SVC

# Create a SVM classifier
#svclassifier = SVC(kernel='linear')
# Train the model
#svclassifier.fit(train_x, train_y)
# Test the model
#y_pred_svm = svclassifier.predict(test_x)

In [0]:
# Print classification_report
#print(classification_report(test_y, y_pred_svm))

In [0]:
# Print confusion_matrix
#print(metrics.confusion_matrix(test_y, y_pred_randomforest))

## 4. Apply PCA, SelectKBest and RFE for feature selections.

### PCA

In [56]:
data3_drop_NoOutputDep = data3_drop1.drop(["OutputDep","OutputArr"], axis=1)
data3_drop_NoOutputDep

,CRSDepTime,CRSArrTime,Distance,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,DayofMonth_2,DayofMonth_3,DayofMonth_4,DayofMonth_5,DayofMonth_6,DayofMonth_7,DayofMonth_8,DayofMonth_9,DayofMonth_10,DayofMonth_11,DayofMonth_12,DayofMonth_13,DayofMonth_14,DayofMonth_15,DayofMonth_16,DayofMonth_17,DayofMonth_18,DayofMonth_19,DayofMonth_20,DayofMonth_21,DayofMonth_22,DayofMonth_23,DayofMonth_24,DayofMonth_25,DayofMonth_26,DayofMonth_27,DayofMonth_28,DayofMonth_29,DayofMonth_30,DayofMonth_31,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7
0,1955,2225,810,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,735,1000,810,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,620,750,515,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,930,1100,515,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,1755,1925,515,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101319,1410,1532,373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
101320,1410,1532,373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
101321,1410,1532,373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
101322,1410,1532,373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


#### Normalize data

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
# create scaler
scaler = MinMaxScaler()

In [59]:
features = ['CRSDepTime', 'CRSArrTime', 'Distance']
x = data3_drop_NoOutputDep.loc[:, features].values
# fit and transform in one step
datanom = scaler.fit_transform(x)
datanom

array([[0.82874618, 0.94468085, 0.31555751],
       [0.08256881, 0.42340426, 0.31555751],
       [0.01223242, 0.31702128, 0.18554429],
       ...,
       [0.49541284, 0.64978723, 0.12296166],
       [0.49541284, 0.64978723, 0.12296166],
       [0.49541284, 0.64978723, 0.12296166]])

In [61]:
# Convert Array to dataframe
feature_convert=['CRSDepTime', 'CRSArrTime', 'Distance']
rows= [x for x in range(100000)]

datascaled_nom=pd.DataFrame(datanom, columns=feature_convert)
datascaled_nom.index=rows
datascaled_nom.head()

,CRSDepTime,CRSArrTime,Distance
0,0.828746,0.944681,0.315558
1,0.082569,0.423404,0.315558
2,0.012232,0.317021,0.185544
3,0.201835,0.465957,0.185544
4,0.706422,0.817021,0.185544


In [63]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
principalComponents=pca.fit_transform(datascaled_nom)
principalDf=pd.DataFrame(data=principalComponents, columns=['principal component 1', 'principal component 2'])

#,'principal component 3','principal component 4','principal component 5',
#'principal component 6','principal component 7'])

principalDf

,principal component 1,principal component 2
0,0.482251,0.093195
1,-0.427745,0.073222
2,-0.543512,-0.063629
3,-0.302814,-0.056905
4,0.311725,-0.043542
...,...,...
99995,0.044398,-0.113384
99996,0.044398,-0.113384
99997,0.044398,-0.113384
99998,0.044398,-0.113384


In [64]:
element_ratio = pca.explained_variance_ratio_
element_ratio

array([0.71469099, 0.24080535])

In [65]:
sum(element_ratio)

0.9554963417015422

#### Apply PCA on Logistic Regression Model

In [0]:
# Train_test_Split
from sklearn.model_selection import train_test_split
train_x_PCA, test_x_PCA, train_y_PCA, test_y_PCA = train_test_split(principalDf, data3_drop1['OutputArr'],
                                                    test_size=0.2, random_state=42)

In [75]:
logisticRegr_PCA = LogisticRegression(class_weight='balanced')
logisticRegr_PCA.fit(train_x_PCA,train_y_PCA)
y_pred_logis_PCA = logisticRegr_PCA.predict(test_x_PCA)

THRESHOLD = 0.8
y_pred_logis_threshold_PCA = np.where(logisticRegr_PCA.predict_proba(test_x_PCA)[:,1] > THRESHOLD, 1, 0)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [76]:
#print classification_report
print(metrics.classification_report(test_y_PCA, y_pred_logis_threshold_PCA))

              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94     17812
         1.0       0.00      0.00      0.00      2188

    accuracy                           0.89     20000
   macro avg       0.45      0.50      0.47     20000
weighted avg       0.79      0.89      0.84     20000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [77]:
#print confusion_matrix
print(metrics.confusion_matrix(test_y_PCA, y_pred_logis_threshold_PCA))

[[17812     0]
 [ 2188     0]]


### SelectKBest

### RFE

## 5. Using gridsearchCV to obtain best parameters for models

### Naive Bayes

### Decision Tree

In [0]:
param_grid_decisiontree = { 
    'max_depth': [1, 5, 10],
    'min_samples_leaf': [1, 10, 15, 20],
}

In [79]:
decision_tree = DecisionTreeClassifier()

decision_tree_grid = GridSearchCV(decision_tree, param_grid = param_grid_decisiontree, cv=10)
# decision_tree_grid = GridSearchCV(d_tree, param_grid, cv=10, scoring='accuracy')

decision_tree_grid.fit(train_x, train_y)

print("tuned hpyerparameters :(best parameters) ",decision_tree_grid.best_params_)
print("accuracy :",decision_tree_grid.best_score_)

tuned hpyerparameters :(best parameters)  {'max_depth': 5, 'min_samples_leaf': 10}
accuracy : 0.96725


In [0]:
# Make predictions using the testing set
y_pred_tree_tune = decision_tree_grid.predict(test_x)

In [81]:
# Print classification_report
print(metrics.classification_report(test_y, y_pred_tree_tune))

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     17812
         1.0       0.84      0.86      0.85      2188

    accuracy                           0.97     20000
   macro avg       0.91      0.92      0.92     20000
weighted avg       0.97      0.97      0.97     20000



The Decision Tree model after fine-tuning is better.

### Logistic Regression

In [0]:
param_grid_log={
    "C":np.logspace(-3,3,7),
    "penalty":["l1","l2"]
    }

In [83]:
logreg_cv=GridSearchCV(LogisticRegression(class_weight='balanced'),param_grid_log,cv=10)

logreg_cv.fit(train_x,train_y)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

tuned hpyerparameters :(best parameters)  {'C': 0.001, 'penalty': 'l1'}
accuracy : 0.967225


In [0]:
# Make predictions using the testing set
y_pred_logis_tune = logreg_cv.predict(test_x)

In [87]:
# Print classification_report
print(metrics.classification_report(test_y, y_pred_logis_tune))

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     17812
         1.0       0.84      0.86      0.85      2188

    accuracy                           0.97     20000
   macro avg       0.91      0.92      0.92     20000
weighted avg       0.97      0.97      0.97     20000



- The Logistic Regression model after fine-tuning is equal to non- tuning model.

### Random Forest

In [0]:
#param_grid_randomforest = { 
#    'n_estimators': [200, 350, 500],
#    'max_features': ['auto', 'sqrt', 'log2'],
#    'max_depth' : [4,6,8],
#    'criterion' :['gini', 'entropy']
#}

param_grid_randomforest = { 
    'n_estimators': [200, 350, 500],
    'max_depth' : [8, 15, 20],
}

In [104]:
random_forest_grid = GridSearchCV(RandomForestClassifier(class_weight="balanced"), param_grid=param_grid_randomforest, cv= 10)

random_forest_grid.fit(train_x, train_y)

print("tuned hpyerparameters :(best parameters) ",random_forest_grid.best_params_)
print("accuracy :",random_forest_grid.best_score_)

tuned hpyerparameters :(best parameters)  {'max_depth': 8, 'n_estimators': 350}
accuracy : 0.9672375


In [0]:
# Make predictions using the testing set
y_pred_randomforest_tune = random_forest_grid.predict(test_x)

In [101]:
# Print classification_report
print(metrics.classification_report(test_y, y_pred_randomforest_tune))

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     17812
         1.0       0.84      0.86      0.85      2188

    accuracy                           0.97     20000
   macro avg       0.91      0.92      0.92     20000
weighted avg       0.97      0.97      0.97     20000



- The Random Forest model after fine-tuning is better.

### Gradient Boosting

n_estimators and Learning rate

In [0]:
param_grid_gradientboosting1 = {
    'learning_rate':[0.85,0.5,0.01],
    'n_estimators':[25,50,75]
}

In [122]:
gradient_grid1 = GridSearchCV(estimator =GradientBoostingClassifier(max_depth=4, min_samples_split=2, min_samples_leaf=1,
                                                            subsample=1,max_features='sqrt', random_state=10), 
            param_grid = param_grid_gradientboosting1, scoring='accuracy',n_jobs=4,iid=False, cv=10)
gradient_grid1.fit(train_x,train_y)
gradient_grid1.best_params_
#gradient_grid1.best_score_

{'learning_rate': 0.5, 'n_estimators': 50}

In [0]:
print("tuned hpyerparameters :(best parameters) ",gradient_grid1.best_params_)

max_depth

In [0]:
param_grid_gradientboosting2 = {
    'max_depth':[1, 2, 4, 6, 8] 
}


In [124]:
gradient_grid2 = GridSearchCV(estimator =GradientBoostingClassifier(learning_rate=0.5,n_estimators=50, min_samples_split=2,
                                                            min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10), 
            param_grid = param_grid_gradientboosting2, scoring='accuracy',n_jobs=4,iid=False, cv=10)
gradient_grid2.fit(train_x,train_y)
gradient_grid2.best_params_
gradient_grid2.best_score_

0.9672249042775375

In [125]:
print("tuned hpyerparameters :(best parameters) ",gradient_grid2.best_params_)

tuned hpyerparameters :(best parameters)  {'max_depth': 1}


In [0]:
# Make predictions using the testing set
y_pred_gradient_tune = gradient_grid2.predict(test_x)

In [127]:
# Print classification_report
print(metrics.classification_report(test_y, y_pred_gradient_tune))

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     17812
         1.0       0.84      0.86      0.85      2188

    accuracy                           0.97     20000
   macro avg       0.91      0.92      0.92     20000
weighted avg       0.97      0.97      0.97     20000



- The Gradient Boosting model after fine-tuning is better.

### SVM

(Slow tuning speed)

In [0]:
#from sklearn.svm import SVC
#def svc_param_selection(X, y, nfolds):
#    Cs = [0.001, 0.01, 0.1, 1, 10]
#    gammas = [0.001, 0.01, 0.1, 1]
#    param_grid = {'C': Cs, 'gamma' : gammas}
#    grid_search = GridSearchCV(SVC(kernel='linear'), param_grid, cv=nfolds)
#    grid_search.fit(X, y)
#    grid_search.best_params_
#    return grid_search.best_params_

In [0]:
#svc_param_selection(train_x,train_y,10)

## 5. Compare performances among models, write up analysis why the model is good or bad in the algorithmic approach

In [0]:
#from sklearn.tree import DecisionTreeClassifier
#from sklearn import model_selection

# prepare configuration for cross validation test harness
#seed = 7
# prepare models
#models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
#results = []
#names = []
#scoring = 'accuracy'
#for name, model in models:
#	kfold = model_selection.KFold(n_splits=10, random_state=seed)
#	cv_results = model_selection.cross_val_score(model, train_x, train_y, cv=kfold, scoring=scoring)
#	results.append(cv_results)
#	names.append(name)
#	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
#	print(msg)
# boxplot algorithm comparison
#fig = plt.figure()
#fig.suptitle('Algorithm Comparison')
#ax = fig.add_subplot(111)
#plt.boxplot(results)
#ax.set_xticklabels(names)
#plt.show()